In [10]:
import numpy as np
import matplotlib.pyplot as plt
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform
from pydrake.multibody.tree import JacobianWrtVariable
plt.rcParams["figure.figsize"] = (13, 7)

In [2]:
model_url = "/home/xuanhien/git/dairlib/examples/trifinger/robot_properties_fingers/urdf/edu/trifingeredu.urdf"
plant = MultibodyPlant(time_step=0)
(trifingeredu, ) = Parser(plant).AddModels(model_url)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"), RigidTransform.Identity())
plant.Finalize()
plant_context = plant.CreateDefaultContext()

In [16]:
position = np.zeros(9)
position[:3] = np.array([0, 0.8, -1.57])
vel = np.zeros(9)
plant.SetPositions(plant_context, position)
plant.SetVelocities(plant_context, vel)

# compute accel, coriolis, and gravity terms.
gravity_torque = plant.CalcGravityGeneralizedForces(plant_context)[:3]
bias_torque = plant.CalcBiasTerm(plant_context)[:3]
mass_matrix = plant.CalcMassMatrix(plant_context)[:3, :3]

## Operational Space Controller formulation

The commanded kinematic target acceleration is $$\ddot{y}_{cmd} = \ddot{y}_{des} + k_p(y_{des} - y) + k_d (\dot{y}_{des} - \dot{y})$$

We can also compute the current acceleration $$\ddot{y} = J \ddot{q} + \dot{J}\dot{q} = J M^{-1} (\tau_g + \tau - \tau_f - C(q, \dot{q})\dot{q}) + \dot{J}\dot{q}$$

OSC forms an optimization problem to track $\ddot{y}_{cmd}$ by minimizing an objective $(\ddot{y}_{cmd} - \ddot{y})^T W (\ddot{y}_{cmd} - \ddot{y})$, subjecting to some constraints

In [19]:
fingertip_0 = plant.GetBodyByName("finger_tip_link_0")
fingertip_0_frame = plant.GetFrameByName("finger_tip_link_0")
J = plant.CalcJacobianTranslationalVelocity(plant_context, JacobianWrtVariable.kQDot, fingertip_0_frame, np.zeros(3), plant.world_frame(), plant.world_frame())[:, :3]

In [37]:
ydd_cmd = J @ np.linalg.inv(mass_matrix) @ (gravity_torque + 0.07 * np.array([-1, 1, 1]))

In [45]:
kp = ydd_cmd / (np.ones(3) * 0.0004)